# Kernels and hyperparameters



This tutorial is an introduction to the syntax used by the optimizer, as well as the principles of Bayesian optimization in general.

We'll start by minimizing Booth's function, which looks like this:

In [ ]:
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt

import bloptools

x1 = x2 = np.linspace(-10, 10, 256)
X1, X2 = np.meshgrid(x1, x2)

plt.pcolormesh(x1, x2, bloptools.experiments.tests.himmelblau(X1, X2), norm=mpl.colors.LogNorm(), shading="auto")
plt.colorbar()
plt.xlabel("x1")
plt.ylabel("x2")


There are several things that our agent will need. The first ingredient is some degrees of freedom (these are always `ophyd` devices) which the agent will move around to different inputs within each DOF's bounds (the second ingredient). We define these here:

In [ ]:
import bloptools

dofs = bloptools.experiments.tests.get_dummy_dofs(2)
bounds = [(-10, 10), (-10, 10)]

The agent automatically samples at different inputs, but we often need some post-processing after data collection. In this case, we need to give the agent a way to compute Himmelblau's function. We accomplish this with a digestion function:

In [ ]:
def digestion(db, uid):

    table = db[uid].table()
    products = {"himmelblau": []}

    for index, entry in table.iterrows():

        products["himmelblau"].append(bloptools.experiments.tests.himmelblau(entry.x1, entry.x2))

    return products

The next ingredient is a task, which gives the agent something to do. We want it to minimize Himmelblau's function, so we make a task that will try to minimize the output of the digestion function called "himmelblau". We also include a transform function, which will make it easier to regress over the outputs of the function.

In [ ]:
from bloptools.tasks import Task

task = Task(key="himmelblau", kind="min", transform=lambda x: np.log(1 + 1e-2 * x))

Combining all of these with a databroker instance, we can make an agent:

In [ ]:
%run -i ../../../examples/prepare_bluesky.py # prepare the bluesky environment

boa = bloptools.bayesian.Agent(
                                dofs=dofs, # things which we move around
                                bounds=bounds, # where we're allowed to move them
                                tasks=task, # tasks for the optimizer
                                digestion=digestion, # how to process the acquisition into task data
                                db=db, # a databroker instance
                                )

RE(boa.initialize(init_scheme='quasi-random', n_init=16))

We initialized the GP with the "quasi-random" strategy, as it doesn't require any prior data. We can view the state of the optimizer's posterior of the tasks over the input parameters:

In [ ]:
boa.plot_tasks()

We can also the agent's posterior about the probability of goodness over the parameters:

We want to learn a bit more, so we can ask the agent where to sample based off of some strategy. Here we use the "esti" strategy, which maximizes the expected sum of tasks improvement.

We can ask the agent to "route" them using ``ortools``, so that we can sample them more quickly if it requires us to e.g. move motors.

In [ ]:
X_to_sample = boa.ask(strategy='esti', n=16, optimize=True, route=True)
plt.scatter(*X_to_sample.T)
plt.plot(*X_to_sample.T)

Let's tell the agent to learn a bit more (it will direct itself):

In [ ]:
RE(boa.learn(strategy='esti', n_iter=1, n_per_iter=4))
boa.plot_tasks()

The agent has updated its model of the tasks, including refitting the hyperparameters. Continuing:

In [ ]:
RE(boa.learn(strategy='esti', n_iter=4, n_per_iter=4))
boa.plot_tasks()

Eventually, we reach a point of saturation where no more improvement takes place:

In [ ]:
RE(boa.learn(strategy='esti', n_iter=4, n_per_iter=4))
boa.plot_tasks()